## <font color='darkblue'>Preface</font>
([article source](https://towardsdatascience.com/swiss-army-knife-of-pandas-filtering-24866166ca97)) <font size='3ptx'><b>Efficient, readable, and a flexible way of pandas filtering</b></font>

Pandas <a href='https://pandas.pydata.org/docs/reference/api/pandas.eval.html'>eval</a> and <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html'>query</a> are some of the most powerful, lesser-known, intuitive functions that are exposed by pandas API. It lets you query the data frame in a readable and intuitive way, different from the traditional way of masking and loc-ing the data. The resulting code is efficient, readable, and flexible.

<b>Filtering a data frame is an important process in any data pipeline and in pandas, it is usually a two-step process.</b> Creating a mask to select the rows and then subsetting the data frame using that mask.

Let’s load in the <b><a href='https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html'>iris dataset</a></b> from scikit-learn,

In [2]:
import pandas as pd
from sklearn import datasets

iris = datasets.load_iris()

In [7]:
iris.data.shape

(150, 4)

In [10]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [14]:
iris_df = pd.DataFrame(iris.data, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [24]:
# target=0 -> setosa, target=1 -> versicolor, target=2->virginica
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [57]:
iris_df['target'] = iris.target
iris_df.sample(n=20)

,sepal_length,sepal_width,petal_length,petal_width,target
145,6.7,3.0,5.2,2.3,2
68,6.2,2.2,4.5,1.5,1
62,6.0,2.2,4.0,1.0,1
91,6.1,3.0,4.6,1.4,1
66,5.6,3.0,4.5,1.5,1
144,6.7,3.3,5.7,2.5,2
114,5.8,2.8,5.1,2.4,2
14,5.8,4.0,1.2,0.2,0
67,5.8,2.7,4.1,1.0,1
111,6.4,2.7,5.3,1.9,2


## <font color='darkblue'>A Simple Example</font>
Let’s say you want to look at rows that are of `species Setosa and sepal-length > 5`. Such multi-column/condition filtering occurs more often in real-world EDA and data pipelines than you can think of. It can be generally written as,

Without query:

In [28]:
iris_df.loc[(iris_df.sepal_length > 5) & (iris_df.target == 0), :].head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
10,5.4,3.7,1.5,0.2,0
14,5.8,4.0,1.2,0.2,0
15,5.7,4.4,1.5,0.4,0


The problem with the above code is, `iris_df` is repeated three times and there are a pair of square brackets and parenthesis. This can be easily rewritten using a query. With query,

In [30]:
iris_df.query("sepal_length > 5 and target == 0").head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
10,5.4,3.7,1.5,0.2,0
14,5.8,4.0,1.2,0.2,0
15,5.7,4.4,1.5,0.4,0


> Pandas query provides a readable, intuitive interface to dataframe filtering that is easier on the eyes and quick to write.

## <font color='darkblue'>Tips for using pandas query</font>
* <font size='3ptx'><b><a href="#sect2_1">Filtering NaN’s</a></b></font>
* <font size='3ptx'><b><a href="#sect2_2">Runtime dynamic substitution using `@`</a></b></font>
* <font size='3ptx'><b><a href="#sect2_3">F-Strings</a></b></font>

Now that we have seen how to use query, we will look at practical tips of unlocking the power of <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html'>query</a> and <a href='https://pandas.pydata.org/docs/reference/api/pandas.eval.html'>eval</a>.

<a id='sect2_1'></a>
### <font color='darkgreen'>Filtering NaN’s</font>
Filtering using NaN is mostly done using <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html'><b>pd</b>.isna</a> and then using it to mask the dataframe.

Without query:

In [31]:
# Set the top 5 rows sepal length as
iris_df.iloc[:5, 0] = None

In [33]:
# Filtering rows that are null
iris_df.loc[pd.isna(iris_df.sepal_length), :]

,sepal_length,sepal_width,petal_length,petal_width,target
0,NaN,3.5,1.4,0.2,0
1,NaN,3.0,1.4,0.2,0
2,NaN,3.2,1.3,0.2,0
3,NaN,3.1,1.5,0.2,0
4,NaN,3.6,1.4,0.2,0


In [35]:
# Filtering rows that are not null
iris_df.loc[~pd.isna(iris_df.sepal_length), :].head()

,sepal_length,sepal_width,petal_length,petal_width,target
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


With query:

In [56]:
# Filtering rows that are not null
iris_df.query("sepal_length == sepal_length").head()

,sepal_length,sepal_width,petal_length,petal_width,target
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


In [37]:
# Filtering rows that are null
iris_df.query("sepal_length != sepal_length")

,sepal_length,sepal_width,petal_length,petal_width,target
0,NaN,3.5,1.4,0.2,0
1,NaN,3.0,1.4,0.2,0
2,NaN,3.2,1.3,0.2,0
3,NaN,3.1,1.5,0.2,0
4,NaN,3.6,1.4,0.2,0


This works because matching `NaN` with itself returns false.

<a id='sect2_2'></a>
### <font color='darkgreen'>Runtime dynamic substitution using `@`</font>
Whenever the comparison value is dynamic in a mask, you can use <b><font color='orange'>@</font></b> to dynamically insert the value into the comparison string.

In [41]:
length_thresh = 5
target = 0

iris_df.query("sepal_length >= @length_thresh and target == @target").head()

,sepal_length,sepal_width,petal_length,petal_width,target
5,5.4,3.9,1.7,0.4,0
7,5.0,3.4,1.5,0.2,0
10,5.4,3.7,1.5,0.2,0
14,5.8,4.0,1.2,0.2,0
15,5.7,4.4,1.5,0.4,0


`length_thresh` and `target` values are resolved dynamically.

In case the column names contain space in between you can use backticks to escape it. e.g.:
```
df.query("`Sepal Length` >= @length_thresh and species == @species")
```

<a id='sect2_3'></a>
### <font color='darkgreen'>F-Strings</font>
Flexibility can be added to the query statement by using f-strings. @ lets you parameterize the right side of an inequality, whereas f-strings will let you parameterized both sides of an inequality.

In [42]:
ltype = "sepal"
length = 5.0

iris_df.query(f"{ltype}_length >= {length}").head()

,sepal_length,sepal_width,petal_length,petal_width,target
5,5.4,3.9,1.7,0.4,0
7,5.0,3.4,1.5,0.2,0
10,5.4,3.7,1.5,0.2,0
14,5.8,4.0,1.2,0.2,0
15,5.7,4.4,1.5,0.4,0


By changing the `ltype` the variable you can control whether `sepal_length` or `petal_length` is used for filtering.

You can also create a complicated query substring and then combine them, making it readable instead of a single complicated combination of filtering.

In [54]:
# Define setosa filter
target, length, ltype = 0, 5.0, "sepal"
setosa_mask = f"target == {target} and {ltype}_length >={length}"
setosa_mask

'target == 0 and sepal_length >=5.0'

In [58]:
# Define virginica filter
target, width, wtype = 2, 1.0, "petal"
virginica_mask = f"target == {target} and {wtype}_width >={width}"
virginica_mask

'target == 2 and petal_width >=1.0'

In [60]:
# combine filter
iris_df.query(f"{setosa_mask} or {virginica_mask}").sample(n=10)

,sepal_length,sepal_width,petal_length,petal_width,target
10,5.4,3.7,1.5,0.2,0
126,6.2,2.8,4.8,1.8,2
35,5.0,3.2,1.2,0.2,0
133,6.3,2.8,5.1,1.5,2
129,7.2,3.0,5.8,1.6,2
127,6.1,3.0,4.9,1.8,2
136,6.3,3.4,5.6,2.4,2
120,6.9,3.2,5.7,2.3,2
18,5.7,3.8,1.7,0.3,0
138,6.0,3.0,4.8,1.8,2


## <font color='darkblue'>Under the hood</font>
Apart from the expressionand inplaceargument, pandas.DataFrame.query accepts `**kwargs` which can take in any argument that you can pass to <a href='https://pandas.pydata.org/docs/reference/api/pandas.eval.html'><b>pandas</b>.eval</a>. Query internally calls Eval to execute the expression.

<a href='https://pandas.pydata.org/docs/reference/api/pandas.eval.html'><b>pandas</b>.eval</a> is akin to python’s <a href='https://docs.python.org/3/library/functions.html#eval'>eval</a> which executes python code in a string.

In [61]:
assert eval("2 + 3 * len('hello')") == 17

Looking at this example, it is dangerously clear that you can execute any python statement inside it.

### <font color='darkgreen'>Eval when not used right is Evil!</font>
With that said, <a href='https://pandas.pydata.org/docs/reference/api/pandas.eval.html'><b>pandas</b>.eval</a> is not as dangerous as it sounds. Unlike python’s <a href='https://docs.python.org/3/library/functions.html#eval'>eval</a> <a href='https://pandas.pydata.org/docs/reference/api/pandas.eval.html'><b>pandas</b>.eval</a> cannot execute arbitrary functions.

In [62]:
eval("print('danger!')")

danger!


In [64]:
# KeyError: 'print'
# pd.eval("print('me')")

It is clear that <a href='https://pandas.pydata.org/docs/reference/api/pandas.eval.html'><b>pandas</b>.eval</a> is similar to python’s <a href='https://docs.python.org/3/library/functions.html#eval'>eval</a> but for a restricted subset of functions and operands.

### <font color='darkgreen'>Arguments</font>
There are other arguments of eval that can be used for more control.

* **Parser**: The eval function accepts an input called `parser` which can be either `python` or `pandas`. Depending on the parser the expression is parsed slightly differently.
* **Engine**: The engine which takes care of executing the expression. By default it is set to `numexpr` which is efficient and faster than the other option which is `python`
* **Local and Global dictionaries**: Sometimes it is useful to pass in variables that are not defined in the current scope inside the expression.

In [74]:
iris_df.query("sepal_length > @thresh", local_dict={'thresh': 5.0}).sample(n=5)

,sepal_length,sepal_width,petal_length,petal_width,target
143,6.8,3.2,5.9,2.3,2
50,7.0,3.2,4.7,1.4,1
136,6.3,3.4,5.6,2.4,2
52,6.9,3.1,4.9,1.5,1
88,5.6,3.0,4.1,1.3,1


## <font color='darkblue'>Just because you can, You shouldn’t!</font>
Pandas eval can also execute statements and assign them to dataframes. For example, you can do this:

In [75]:
iris_df.eval("length_ratio = sepal_length / petal_length", inplace=True)

In [76]:
iris_df.sample(n=5)

,sepal_length,sepal_width,petal_length,petal_width,target,length_ratio
145,6.7,3.0,5.2,2.3,2,1.288462
10,5.4,3.7,1.5,0.2,0,3.600000
97,6.2,2.9,4.3,1.3,1,1.441860
120,6.9,3.2,5.7,2.3,2,1.210526
65,6.7,3.1,4.4,1.4,1,1.522727


The above statement will create a column `length_ratio` in the dataframe. This is the opposite of why we would want to use <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html'>query</a> statement. <font color='darkred'><b>Unlike <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html'>query</a> expression which improves readability, this type of usage is ambiguous and hinders readability.</b></font>